# SBML import, observation model, sensitivity analysis, data export and visualization

This is an example using the [model_steadystate_scaled.xml] model to demonstrate:

* SBML import
* specifying the observation model
* performing sensitivity analysis
* exporting and visualizing simulation results

In [ ]:
# SBML model we want to import
sbml_file = "model_steadystate_scaled_without_observables.xml"
# Name of the model that will also be the name of the python module
model_name = "model_steadystate_scaled"
# Directory to which the generated model code is written
model_output_dir = model_name

import amici
import libsbml
import matplotlib.pyplot as plt
import numpy as np
from amici.sim.sundials import (
    ExpData,
    ParameterScaling,
    SensitivityMethod,
    SensitivityOrder,
    run_simulation,
)

## The example model

Here we use `libsbml` to show the reactions and species described by the model (this is independent of AMICI).

In [ ]:
sbml_reader = libsbml.SBMLReader()
sbml_doc = sbml_reader.readSBML(sbml_file)
sbml_model = sbml_doc.getModel()
dir(sbml_doc)

print("Species: ", [s.getId() for s in sbml_model.getListOfSpecies()])

print("\nReactions:")
for reaction in sbml_model.getListOfReactions():
    reactants = " + ".join(
        [
            "{} {}".format(
                int(r.getStoichiometry()) if r.getStoichiometry() > 1 else "",
                r.getSpecies(),
            )
            for r in reaction.getListOfReactants()
        ]
    )
    products = " + ".join(
        [
            "{} {}".format(
                int(r.getStoichiometry()) if r.getStoichiometry() > 1 else "",
                r.getSpecies(),
            )
            for r in reaction.getListOfProducts()
        ]
    )
    reversible = "<" if reaction.getReversible() else ""
    print(
        "%3s: %10s %1s->%10s\t\t[%s]"  # noqa: UP031
        % (
            reaction.getId(),
            reactants,
            reversible,
            products,
            libsbml.formulaToL3String(reaction.getKineticLaw().getMath()),
        )
    )

## Importing an SBML model, compiling and generating an AMICI module

Before we can use AMICI to simulate our model, the SBML model needs to be translated to C++ code. This is done by `amici.SbmlImporter`.

In [ ]:
# Create an SbmlImporter instance for our SBML model
sbml_importer = amici.SbmlImporter(sbml_file)

In this example, we want to specify fixed parameters, observables and a $\sigma$ parameter. Unfortunately, the latter two are not part of the [SBML standard](https://sbml.org/). However, they can be provided to `amici.SbmlImporter.sbml2amici` as demonstrated in the following.

### Fixed parameters

Fixed parameters, i.e. parameters with respect to which no sensitivities are to be computed (these are often parameters specifying a certain experimental condition) are provided as a list of parameter names.

In [ ]:
fixed_parameters = ["k0"]

### Observation model

Specifying the observation model (i.e., the quantities that are observed, as well as the respective error models) is beyond the scope of SBML. Here we define that manually.

If you are looking for a more scalable way of defining observables, then checkout [PEtab](https://github.com/PEtab-dev/PEtab). Another possibility is using SBML's [`AssignmentRule`s](https://sbml.org/software/libsbml/5.18.0/docs/formatted/python-api/classlibsbml_1_1_assignment_rule.html) to specify model outputs within the SBML file.



For model import in AMICI, the different types of measurements are represented as `MeasurementChannels`.
The measurement channel is characterized by an ID, an optional name, the observation function (`MeasurementChannels.formula`), the type of noise distribution (`MeasurementChannels.noise_distribution`, defaults to a normal distribution), and the scale parameter of that distribution (`MeasurementChannels.sigma`).
The symbols used in the observation function and for the scale parameter must already be defined in the model.

In [ ]:
# Define observation model
from amici import MeasurementChannel as MC

observation_model = [
    MC(id_="observable_x1", formula="x1"),
    MC(id_="observable_x2", formula="x2"),
    MC(id_="observable_x3", formula="x3"),
    MC(id_="observable_x1_scaled", formula="scaling_x1 * x1"),
    MC(id_="observable_x2_offsetted", formula="offset_x2 + x2"),
    MC(
        id_="observable_x1withsigma",
        formula="x1",
        sigma="observable_x1withsigma_sigma",
    ),
]

### Generating the module

Now we can generate the python module for our model. `amici.SbmlImporter.sbml2amici` will symbolically derive the sensitivity equations, generate C++ code for model simulation, and assemble the python module. Standard logging verbosity levels can be passed to this function to see timestamped progression during code generation.

In [ ]:
import logging

model = sbml_importer.sbml2amici(
    model_name,
    model_output_dir,
    verbose=logging.INFO,
    observation_model=observation_model,
    fixed_parameters=fixed_parameters,
)

### Importing the module and loading the model

A model instance for simulation is returned by `sbml2amici`, but we can also import the module containing the model class later using `amici.import_model_module`:

In [ ]:
model_module = amici.import_model_module(model_name, model_output_dir)

And get an instance of our model from which we can retrieve information such as parameter names:

In [ ]:
model = model_module.get_model()

print("Model name:            ", model.get_name())
print("Model parameters:      ", model.get_free_parameter_ids())
print("Model outputs:         ", model.get_observable_ids())
print("Model state variables: ", model.get_state_ids())

## Running simulations and analyzing results

After importing the model, we can run simulations using `amici.run_simulation`. This requires a `Model` instance and a `Solver` instance. Optionally you can provide measurements inside an `ExpData` instance, as shown later in this notebook.

In [ ]:
# Create Model instance
model = model_module.get_model()

# set timepoints for which we want to simulate the model
model.set_timepoints(np.linspace(0, 60, 60))

# Create solver instance
solver = model.create_solver()

# Run simulation using default model parameters and solver options
rdata = run_simulation(model, solver)

In [ ]:
print(
    "Simulation was run using model default parameters as specified in the SBML model:"
)
print(dict(zip(model.get_free_parameter_ids(), model.get_free_parameters())))

Simulation results are provided as `numpy.ndarray`s in the returned dictionary:

In [ ]:
# np.set_printoptions(threshold=8, edgeitems=2)
for key, value in rdata.items():
    print(f"{key:12s}", value)

In [ ]:
# In particular for interactive use, ReturnDataView.by_id() and amici.evaluate provides a more convenient way to access slices of the result:
# Time trajectory of observable observable_x1
print(f"{rdata.by_id('observable_x1')=}")
# Time trajectory of state variable x2
print(f"{rdata.by_id('x2')=}")

Alternatively, those data can be accessed through `ReturnData.xr.*` as [xarray.DataArray](https://docs.xarray.dev/en/stable/index.html) objects, that contain additional metadata such as timepoints and identifiers. This allows for more convenient indexing and plotting of the results.

In [ ]:
rdata.xr.x

In [ ]:
rdata.xr.x.to_pandas()

### Plotting trajectories

The simulation results above did not look too appealing. Let's plot the trajectories of the model states and outputs  them using `matplotlib.pyplot`:

In [ ]:
from amici.sim.sundials.plotting import *

plot_state_trajectories(rdata)
plot_observable_trajectories(rdata)

We can also evaluate symbolic expressions of model quantities using `amici.numpy.evaluate`, or directly plot the results using  `amici.plotting.plot_expressions`:

In [ ]:
plot_expressions("observable_x1 + observable_x2 + observable_x3", rdata=rdata)

### Computing likelihood

Often model parameters need to be inferred from experimental data. This is commonly done by maximizing the likelihood of observing the data given to current model parameters. AMICI will compute this likelihood if experimental data is provided to `run_amici_simulations` as optional third argument. Measurements along with their standard deviations are provided through an `ExpData` instance.

In [ ]:
# Create model instance and set time points for simulation
model = model_module.get_model()
model.set_timepoints(np.linspace(0, 10, 11))

# Create solver instance, keep default options
solver = model.create_solver()

# Run simulation without experimental data
rdata = run_simulation(model, solver)

# Create ExpData instance from simulation results
edata = ExpData(rdata, 1.0, 0.0)

# Re-run simulation, this time passing "experimental data"
rdata = run_simulation(model, solver, edata)

print(f"Log-likelihood {rdata['llh']:f}")

The provided measurements can be visualized together with the simulation results by passing the `ExpData` to `amici.plotting.plot_observable_trajectories`:

In [ ]:
plot_observable_trajectories(rdata, edata=edata)
plt.legend(loc="center left", bbox_to_anchor=(1.04, 0.5))

### Simulation tolerances
Numerical error tolerances are often critical to get accurate results. For the state variables, integration errors can be controlled using `set_relative_tolerance` and `set_absolute_tolerance`. Similar functions exist for sensitivities, steady states and quadratures. We initially compute a reference solution using extremely low tolerances and then assess the influence on integration error for different levels of absolute and relative tolerance.

In [ ]:
solver.set_relative_tolerance(1e-16)
solver.set_absolute_tolerance(1e-16)
solver.set_sensitivity_order(SensitivityOrder.none)
rdata_ref = run_simulation(model, solver, edata)


def get_simulation_error(solver):
    rdata = run_simulation(model, solver, edata)
    return np.mean(np.abs(rdata["x"] - rdata_ref["x"])), np.mean(
        np.abs(rdata["llh"] - rdata_ref["llh"])
    )


def get_errors(tolfun, tols):
    solver.set_relative_tolerance(1e-16)
    solver.set_absolute_tolerance(1e-16)
    x_errs = []
    llh_errs = []
    for tol in tols:
        getattr(solver, tolfun)(tol)
        x_err, llh_err = get_simulation_error(solver)
        x_errs.append(x_err)
        llh_errs.append(llh_err)
    return x_errs, llh_errs


atols = np.logspace(-5, -15, 100)
atol_x_errs, atol_llh_errs = get_errors("set_absolute_tolerance", atols)

rtols = np.logspace(-5, -15, 100)
rtol_x_errs, rtol_llh_errs = get_errors("set_relative_tolerance", rtols)

fig, axes = plt.subplots(1, 2, figsize=(15, 5))


def plot_error(tols, x_errs, llh_errs, tolname, ax):
    ax.plot(tols, x_errs, "r-", label="x")
    ax.plot(tols, llh_errs, "b-", label="llh")
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlabel(f"{tolname} tolerance")
    ax.set_ylabel("average numerical error")
    ax.legend()


plot_error(atols, atol_x_errs, atol_llh_errs, "absolute", axes[0])
plot_error(rtols, rtol_x_errs, rtol_llh_errs, "relative", axes[1])

# reset relative tolerance to default value
solver.set_relative_tolerance(1e-8)
solver.set_absolute_tolerance(1e-16)

## Sensitivity analysis

AMICI can provide first- and second-order sensitivities using the forward- or adjoint-method. The respective options are set on the Model and Solver objects.

### Forward sensitivity analysis

In [ ]:
model = model_module.get_model()
model.set_timepoints(np.linspace(0, 10, 11))

# sensitivities w.r.t. all parameters
model.require_sensitivities_for_all_parameters()
# sensitivities w.r.t. the specified parameters:
# model.set_parameter_list([1, 2])

# parameters are used as-is (not log-transformed)
model.set_parameter_scale(ParameterScaling.none)

solver = model.create_solver()
# forward sensitivity analysis
solver.set_sensitivity_method(SensitivityMethod.forward)
# first-order sensitivities
solver.set_sensitivity_order(SensitivityOrder.first)

rdata = run_simulation(model, solver)

# print sensitivity-related results
for key, value in rdata.items():
    if key.startswith("s"):
        print(f"{key:12s}", value)

## Adjoint sensitivity analysis

In [ ]:
# Set model options
model = model_module.get_model()
p_orig = np.array(model.get_free_parameters())
p_orig[
    list(model.get_free_parameter_ids()).index("observable_x1withsigma_sigma")
] = 0.1  # Change default parameter
model.set_free_parameters(p_orig)
model.set_parameter_scale(ParameterScaling.none)
model.set_timepoints(np.linspace(0, 10, 21))

solver = model.create_solver()
solver.set_max_steps(10**4)  # Set maximum number of steps for the solver

# simulate time-course to get artificial data
rdata = run_simulation(model, solver)
edata = ExpData(rdata, 1.0, 0)
edata.fixed_parameters = model.get_fixed_parameters()
# set sigma to 1.0 except for observable 5, so that p[7] is used instead
# (if we have sigma parameterized, the corresponding ExpData entries must NaN, otherwise they will override the parameter)
edata.set_noise_scales(
    rdata["t"] * 0 + np.nan,
    list(model.get_observable_ids()).index("observable_x1withsigma"),
)

# enable sensitivities
# First-order ...
solver.set_sensitivity_order(SensitivityOrder.first)
# ... adjoint sensitivities
solver.set_sensitivity_method(SensitivityMethod.adjoint)
# ... w.r.t. all parameters
model.require_sensitivities_for_all_parameters()

# compute adjoint sensitivities
rdata = run_simulation(model, solver, edata)
# print(rdata['sigmay'])
print(f"Log-likelihood: {rdata['llh']}")
print(f"Gradient: {rdata['sllh']}")

## Finite differences gradient check

Compare AMICI-computed gradient with finite differences

In [ ]:
from scipy.optimize import check_grad


def func(x0, symbol="llh", x0full=None, plist=[], verbose=False):
    p = x0[:]
    if len(plist):
        p = x0full[:]
        p[plist] = x0
    verbose and print(f"f: p={p}")

    old_parameters = model.get_free_parameters()
    solver.set_sensitivity_order(SensitivityOrder.none)
    model.set_free_parameters(p)
    rdata = run_simulation(model, solver, edata)

    model.set_free_parameters(old_parameters)

    res = np.sum(rdata[symbol])
    verbose and print(res)
    return res


def grad(x0, symbol="llh", x0full=None, plist=[], verbose=False):
    p = x0[:]
    if len(plist):
        model.set_parameter_list(plist)
        p = x0full[:]
        p[plist] = x0
    else:
        model.require_sensitivities_for_all_parameters()
    verbose and print(f"g: p={p}")

    old_parameters = model.get_free_parameters()
    solver.set_sensitivity_method(SensitivityMethod.forward)
    solver.set_sensitivity_order(SensitivityOrder.first)
    model.set_free_parameters(p)
    rdata = run_simulation(model, solver, edata)

    model.set_free_parameters(old_parameters)

    res = rdata[f"s{symbol}"]
    if not isinstance(res, float):
        if len(res.shape) == 3:
            res = np.sum(res, axis=(0, 2))
    verbose and print(res)
    return res


epsilon = 1e-4
err_norm = check_grad(func, grad, p_orig, "llh", epsilon=epsilon)
print(f"sllh: |error|_2: {err_norm:f}")
# assert err_norm < 1e-6
print()

for ip in range(model.np()):
    plist = [ip]
    p = p_orig.copy()
    err_norm = check_grad(
        func, grad, p[plist], "llh", p, [ip], epsilon=epsilon
    )
    print(f"sllh: p[{ip:d}]: |error|_2: {err_norm:f}")

print()
for ip in range(model.np()):
    plist = [ip]
    p = p_orig.copy()
    err_norm = check_grad(func, grad, p[plist], "y", p, [ip], epsilon=epsilon)
    print(f"sy: p[{ip}]: |error|_2: {err_norm:f}")

print()
for ip in range(model.np()):
    plist = [ip]
    p = p_orig.copy()
    err_norm = check_grad(func, grad, p[plist], "x", p, [ip], epsilon=epsilon)
    print(f"sx: p[{ip}]: |error|_2: {err_norm:f}")

print()
for ip in range(model.np()):
    plist = [ip]
    p = p_orig.copy()
    err_norm = check_grad(
        func, grad, p[plist], "sigmay", p, [ip], epsilon=epsilon
    )
    print(f"ssigmay: p[{ip}]: |error|_2: {err_norm:f}")

In [ ]:
eps = 1e-4
op = model.get_free_parameters()


solver.set_sensitivity_method(
    SensitivityMethod.forward
)  # forward sensitivity analysis
solver.set_sensitivity_order(
    SensitivityOrder.first
)  # first-order sensitivities
model.require_sensitivities_for_all_parameters()
solver.set_relative_tolerance(1e-12)
rdata = run_simulation(model, solver, edata)


def fd(x0, ip, eps, symbol="llh"):
    p = list(x0[:])
    old_parameters = model.get_free_parameters()
    solver.set_sensitivity_order(SensitivityOrder.none)
    p[ip] += eps
    model.set_free_parameters(p)
    rdata_f = run_simulation(model, solver, edata)
    p[ip] -= 2 * eps
    model.set_free_parameters(p)
    rdata_b = run_simulation(model, solver, edata)

    model.set_free_parameters(old_parameters)
    return (rdata_f[symbol] - rdata_b[symbol]) / (2 * eps)


def plot_sensitivities(symbol, eps):
    fig, axes = plt.subplots(4, 2, figsize=(15, 10))
    for ip in range(4):
        fd_approx = fd(model.get_free_parameters(), ip, eps, symbol=symbol)

        axes[ip, 0].plot(
            edata.get_timepoints(), rdata[f"s{symbol}"][:, ip, :], "r-"
        )
        axes[ip, 0].plot(edata.get_timepoints(), fd_approx, "k--")
        axes[ip, 0].set_ylabel(f"sensitivity {symbol}")
        axes[ip, 0].set_xlabel("time")

        axes[ip, 1].plot(
            edata.get_timepoints(),
            np.abs(rdata[f"s{symbol}"][:, ip, :] - fd_approx),
            "k-",
        )
        axes[ip, 1].set_ylabel("difference to fd")
        axes[ip, 1].set_xlabel("time")
        axes[ip, 1].set_yscale("log")

    plt.tight_layout()
    plt.show()

In [ ]:
plot_sensitivities("x", eps)

In [ ]:
plot_sensitivities("y", eps)